# Dev Jokes ASCII Art AI Generator

This script generates ASCII art of developer jokes using Azure OpenAI Dall·e and the `ascii_magic` library.

In [ ]:
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
import json

from ascii_magic import AsciiArt

# Data preparation

We need to remove some quotes and other strange characters before generating the pictures.

In [ ]:
import pandas as pd

df = pd.read_excel("developer_jokes_3.xlsx")
df

In [ ]:
df['Joke'][3]

In [ ]:
# find non ascii characters on dataframe
non_ascii_chars = set(char for joke in df['Joke'] for char in str(joke) if ord(char) > 127)
print(sorted(non_ascii_chars))

Not a big deal, we can live with that.

In [ ]:
endpoint = os.getenv(
    "AZURE_OPENAI_ENDPOINT", "https://cog-7qxr3pddyatvm.openai.azure.com/"
)
api_version = os.getenv("OPENAI_API_VERSION", "2024-04-01-preview")
dalle_deployment = os.getenv("DALL_E_DEPLOYMENT_NAME", "dall-e-3")
gpt_deployment = os.getenv("GPT_DEPLOYMENT_NAME", "gpt-4.1-nano")

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
)

In [ ]:
import time
from IPython.display import Image, display, clear_output
import requests
from urllib.parse import urlparse


def generate_image_from_joke(joke:str)->str:
    messages = [
        {
            "role": "system",
            "content": "You are a helpful designer that ideates images based on jokes. The images are abstract ideas of the joke. They will be converted to ASCII art, so don't overdo it or add any text, as it won't be visible once converted.",
        },
        {
            "role": "user",
            "content": "Joke: %s\nIdea: " % joke,
        },
    ]

    # Generate the description of the image based on the joke
    print(f"Generating description for joke: {joke}")
    completion = client.chat.completions.create(
        model=gpt_deployment,
        messages=messages,
        max_tokens=800,
        temperature=1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )

    print(f"Description: {completion.choices[0].message.content}")

    result = client.images.generate(
        model=dalle_deployment,
        prompt=completion.choices[0].message.content,
        n=1,
        style="vivid",
        quality="standard",
    )
    image_url = json.loads(result.model_dump_json())["data"][0]["url"]
    # download the image
    if not image_url:
        raise ValueError("No image URL returned from the model.")

    return image_url

try:
    for index,row in df.iterrows():
        idx=str(index).zfill(4)
        fname = f"art/ascii_art_{idx}.html"        
        image_filename=f"images/image_{idx}.png"

        if os.path.exists(f"../docs/{fname}"):
            print(f"HTML file already exists: {fname}")        
        else:
            if os.path.exists(image_filename):
                print(f"Image already exists: {image_filename}")            
            else:
                retry_count = 0
                while retry_count < 3:
                    try:
                        image_url = generate_image_from_joke(row["Joke"])
                        break
                    except Exception as e:
                        print(f"Error generating image: {e}")
                        retry_count += 1
                        time.sleep(5 ** retry_count + 1)  # Exponential backoff
                        if retry_count >= 3:
                            print("Failed to generate image after 3 attempts.")
                            continue

                # Download the image
                print(f"Downloading image from URL: {image_url}")
                response = requests.get(image_url)
                if response.status_code != 200:
                    raise ValueError(
                        f"Failed to download image, status code: {response.status_code}"
                    )

                with open(image_filename, "wb") as f:
                    f.write(response.content)

            ascii_art = AsciiArt.from_image(image_filename)

            html=ascii_art.to_html()       
            with open(f"../docs/{fname}", "w") as f:
                f.write(html)

        df.at[index, "ascii_art"] = fname
        df.to_json("../docs/developer_jokes.json", orient="records", indent=2)
finally:    
    df.to_json("../docs/developer_jokes.json", orient="records", indent=2)

In [ ]:
df.head(10)
